In [5]:
import pandas as pd
import numpy as np
import re,json

import pyarrow as pa

from collections import defaultdict,Counter

from sklearn.feature_extraction.text import TfidfVectorizer

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

### 테스트 데이터 load

In [66]:

tmp_data = [[153709120,'갤럭시 중고폰'],[153709376,'연예인 지갑 지갑'],[153710656,'갤럭시s10 갤럭시북 NT950XDZ-G58AW 중고폰']]
tmp = pd.DataFrame(tmp_data, columns=['id','name'])

tmp.head()

,id,name
0,153709120,갤럭시 중고폰
1,153709376,연예인 지갑 지갑
2,153710656,갤럭시s10 갤럭시북 NT950XDZ-G58AW 중고폰


## 1. 토큰화

In [67]:
def tokenizer(data):
    #token=[]
    token = ''

    data = data.lower() #소문자로 변환

    words = data.split() #공백으로 분리
    #print("split ", words)

    #규칙에 해당 -> findall
    '''
    - 공백 기준으로 분리
    - 영어는 소문자로 변환
    - 공백으로 분리된 텍스트 안에서 다음 규칙으로 토큰을 구분
    - 연속된 한글 : [가-힣]+
    - 연속된 자모 : [ㄱ-ㅎ|ㅏ-ㅣ]+
    - 연속된 영문, 숫자, 하이픈(-) : [a-zA-Z0-9-]+
    - 그 외 문자는 묶어서 하나로 취급 : [^ A-Za-z0-9가-힣+] (연속된 특수문자)
    '''
    p = re.compile("[가-힣]+|[ㄱ-ㅎ|ㅏ-ㅣ]+|[a-z0-9-]+|[^ a-z0-9가-힣+]") #규칙

    for word in words:
        find = re.findall(p,word)
        for w in find:
            token=token+w+' '
            #token.append(w)
    return token

In [68]:
tmp['token']=tmp['name'].apply(tokenizer)
tmp.head()

,id,name,token
0,153709120,갤럭시 중고폰,갤럭시 중고폰
1,153709376,연예인 지갑 지갑,연예인 지갑 지갑
2,153710656,갤럭시s10 갤럭시북 NT950XDZ-G58AW 중고폰,갤럭시 s10 갤럭시북 nt950xdz-g58aw 중고폰


## 2.inverted index

In [69]:
js = tmp.to_json(orient = 'records')
json_data =json.loads(js)
json_data

[{'id': 153709120, 'name': '갤럭시 중고폰', 'token': '갤럭시 중고폰 '},
 {'id': 153709376, 'name': '연예인 지갑 지갑', 'token': '연예인 지갑 지갑 '},
 {'id': 153710656,
  'name': '갤럭시s10 갤럭시북 NT950XDZ-G58AW 중고폰',
  'token': '갤럭시 s10 갤럭시북 nt950xdz-g58aw 중고폰 '}]

In [70]:

index_dict=defaultdict(list)

for data in json_data:
    for token in data['token'].split():
            index_dict[token].append(data['id'])

print(index_dict)

p=pd.DataFrame(list(index_dict.items()),columns=['token', 'docu_list'])
#p=p.set_index('token')


defaultdict(<class 'list'>, {'갤럭시': [153709120, 153710656], '중고폰': [153709120, 153710656], '연예인': [153709376], '지갑': [153709376, 153709376], 's10': [153710656], '갤럭시북': [153710656], 'nt950xdz-g58aw': [153710656]})


In [71]:
import pyarrow.parquet as pq
table = pa.Table.from_pandas(p)
pq.write_table(table, 'example.parquet')

In [72]:
index_df = pq.read_table('example.parquet').to_pandas()
index_df.head()

,token,docu_list
0,갤럭시,"[153709120, 153710656]"
1,중고폰,"[153709120, 153710656]"
2,연예인,[153709376]
3,지갑,"[153709376, 153709376]"
4,s10,[153710656]


## 3.tf-idf 적용 (TfidfVectorizer )

In [73]:
## token 많아질수록 벡터의 차원이 커지는 문제
## 가장 많이 나온 단어 n개만 사용하는 max_features 파라미터 : TfidfVectorizer(max_features=4)
## all_teokn = tmp['token ] : list

words=tmp['token']
#print(words)

vect2 = TfidfVectorizer()
tfvect_matrix = vect2.fit_transform(words)
tfvect_matrix.toarray()

array([[0.        , 0.        , 0.        , 0.70710678, 0.        ,
        0.        , 0.70710678, 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.4472136 , 0.        , 0.89442719],
       [0.44036207, 0.44036207, 0.44036207, 0.3349067 , 0.44036207,
        0.        , 0.3349067 , 0.        ]])

In [15]:
token_tf = vect2.get_feature_names() #유니크한 단어수
token_tf

['g58aw', 'nt950xdz', 's10', '갤럭시', '갤럭시북', '연예인', '중고폰', '지갑']

In [74]:
## tf-idf적용한 결과 df로 생성 

tfidv_df = pd.DataFrame(tfvect_matrix.toarray(), columns = sorted(token_tf))
tfidv_df.index = tmp['id']
tfidv_df

,g58aw,nt950xdz,s10,갤럭시,갤럭시북,연예인,중고폰,지갑
id,,,,,,,,
153709120,0.000000,0.000000,0.000000,0.707107,0.000000,0.000000,0.707107,0.000000
153709376,0.000000,0.000000,0.000000,0.000000,0.000000,0.447214,0.000000,0.894427
153710656,0.440362,0.440362,0.440362,0.334907,0.440362,0.000000,0.334907,0.000000


## 4. 검색어(query) 테스트

- 입력한 키워드 토큰화 리스트 : q_token
- reverted_index로 입력한 quey가 있는 문서id 리스트 리턴  
- 문서들의 교집합을 찾음 : q_documents 
<br><br>
- 교집합 문서들의 name : search_dc
- 교집합 문서들의 td-idf(score) : search_tf
- search_dc와 search_tf merge : search
- score기준 내림차순 정렬 
<br><br>
- 최종결과 response message

In [115]:
q = "아이폰 중고폰"
#입력한 키워드 토큰화 
token_list = tokenizer(q).split()
print(token_list)

['아이폰', '중고폰']


In [55]:
# index_dict 그대로 사용한 ver 

# query가 들어있는 문서id
q_documents=[]
for tk in token_list:
    print(index_dict[tk])
    q_documents.append(set(index_dict[tk])) 
print(q_documents)

# 문서들의 교집합 
query_documents = list(q_documents[0].intersection(*q_documents))
print(query_documents)

[153679680, 153682496]
[153679680, 153682496]
[{153679680, 153682496}, {153679680, 153682496}]
[153679680, 153682496]


In [117]:
# 0831 : ndex_dict 파일로 바꾼 ver - 없는 키워드 입력 고려 

q_documents=[]
search_token =index_df[index_df['token'].isin(token_list)]
print(search_token)

new_token_list = list(search_token['token'])
print(new_token_list) #없는 키워드제거 된 list 

if len(new_token_list) == 0:
    print("없는 키워드")

for docu_list in search_token['docu_list']:
    q_documents.append(set(docu_list))

# 문서들의 교집합 
query_documents = list(q_documents[0].intersection(*q_documents))

print(query_documents)

if len(query_documents) == 0:
    print("교집합없음")

1
[153709120, 153710656]


In [113]:
#교집합 문서들의 id,name
search_dc = tmp[tmp['id'].isin(query_documents)]
search_dc=search_dc.set_index('id')
search_dc

#교집합 문서들에 대해서 tf-dif(score값)
#tfidv_df.loc[query_documents] #교집합문서
search_tf = tfidv_df.loc[query_documents][new_token_list] ## 여기에서 token_list에서 tokne에 없는게 있음 

search_tf['score'] = search_tf.sum(axis=1)
search_tf

# search_dc와 search_tf join (by id)
search=search_tf.join(search_dc,how='inner')
search['pid']=search.index
search.sort_values(by=['score'],ascending=[False],inplace=True) #score기준 정렬
search

,name,token
id,,
153710656,갤럭시s10 갤럭시북 NT950XDZ-G58AW 중고폰,갤럭시 s10 갤럭시북 nt950xdz-g58aw 중고폰


,갤럭시,중고폰,s10,score
id,,,,
153710656,0.334907,0.334907,0.440362,1.110175


,갤럭시,중고폰,s10,score,name,token,pid
id,,,,,,,
153710656,0.334907,0.334907,0.440362,1.110175,갤럭시s10 갤럭시북 NT950XDZ-G58AW 중고폰,갤럭시 s10 갤럭시북 nt950xdz-g58aw 중고폰,153710656


In [114]:
# 최종결과 
response = search[['pid','name','score']]
response

# response msg
js = response.to_json(orient='records')
res_data =json.loads(js)
res_data

,pid,name,score
id,,,
153710656,153710656,갤럭시s10 갤럭시북 NT950XDZ-G58AW 중고폰,1.110175


[{'pid': 153710656,
  'name': '갤럭시s10 갤럭시북 NT950XDZ-G58AW 중고폰',
  'score': 1.1101754726}]